# Eyepoint_returns

Customer is preparing for the commercial launch of two ophthalmic therapies:
1. DEXYCU: A dropless long-acting therapeutic for treating inflammation associated with cataract surgery.
2. YUTIQ: A long-acting therapeutic delivered via the Durasert bioerodible technology platform for treatment of posterior segment uveitis.

To support the upcoming launch, Customer is seeking a data and analytics partner to provide robust insights into the target
patient and provider populations to support salesforce optimization and market access priorities.

# Connections

## Importing required modules

In [9]:
## importing packages and setting connector to snowflake
import qgrid
import pandas as pd
from drg_connect import Snowflake
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [10]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW', database='PROJECT_ANALYTICS',schema ='EYEPOINT_PHYFACTAR')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Uploading Cataract codes table

In [14]:
#Read the .csv file for procedure codes
cat_proc_list = pd.read_csv('Cataract_proc_codes.csv', dtype=str)
cat_proc_list.head()

proc_code
0     66940
1     66920
2     66982
3     66983
4     66984

In [16]:
#upload code_list in eyepoint
snow.upload_dataframe(cat_proc_list, 'cat_proc_codes', schema = 'EYEPOINT_PHYFACTAR')

Upload successful!


# RAVEN PROCEDURE
Getting data from procedure table based on procedure codes of interest

In [86]:
%%read_sql
DROP TABLE IF EXISTS Eyepoint_cataract_raven_med;
CREATE TABLE Eyepoint_cataract_raven_med as
      SELECT CLAIM_NUMBER,
             ENCRYPTED_KEY_1 as patient_suffix,
             ENCRYPTED_KEY_2,
             case when ENCRYPTED_KEY_2 is null then ENCRYPTED_KEY_1 else ENCRYPTED_KEY_1||ENCRYPTED_KEY_2 end as Patient_ID,
             PROCEDURE,
             YEAR_OF_SERVICE,
             PLACE_SERVICE,
             NDC
        FROM RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
       WHERE PROCEDURE IN ('66940','66920','66982','66983','66984','G9389',
                           'G9390','66840','66850','66852','65175','66986',
                           '13.71','13.41','13.42','13.43','13.6','13.69',
                           '1371','1341','1342','1343','136','1369')
             AND YEAR_OF_SERVICE >= '2017-06-01'
             AND YEAR_OF_SERVICE <= '2018-05-31'
             AND encrypted_key_1 IS NOT NULL 
             AND upper(encrypted_key_1) <> 'NULL'
             AND encrypted_key_1 <> '' 
             AND left(encrypted_key_1,3) <> 'XXX';

Query started at 07:13:53 PM India Standard Time; Query executed in 0.04 mQuery started at 07:13:55 PM India Standard Time; Query executed in 6.44 m

status
0  Table EYEPOINT_CATARACT_RAVEN_MED successfully...

In [20]:
%%read_sql

CREATE or replace TABLE Dex_cataract_raven_proc as
      SELECT CLAIM_NUMBER,
             ENCRYPTED_KEY_1 as patient_suffix,
             ENCRYPTED_KEY_2,
             case when ENCRYPTED_KEY_2 is null then ENCRYPTED_KEY_1 else ENCRYPTED_KEY_1||ENCRYPTED_KEY_2 end as Patient_ID,
             PROCEDURE,
             PROCEDURE_MODIFIER_1,
             PROCEDURE_MODIFIER_2,
             PROCEDURE_MODIFIER_3,
             PROCEDURE_MODIFIER_4,           
             YEAR_OF_SERVICE,
             PLACE_SERVICE
        FROM RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
       WHERE upper(PROCEDURE) IN (SELECT distinct proc_code 
                                    FROM cat_proc_codes)
             AND YEAR_OF_SERVICE >= '2017-06-01'
             AND YEAR_OF_SERVICE <= '2018-05-31'
             AND encrypted_key_1 IS NOT NULL 
             AND upper(encrypted_key_1) <> 'NULL'
             AND encrypted_key_1 <> '' 
             AND left(encrypted_key_1,3) <> 'XXX';

Query started at 01:59:37 PM India Standard Time; Query executed in 13.43 m

status
0  Table DEX_CATARACT_RAVEN_PROC successfully cre...

In [23]:
%%read_sql

SELECT PROCEDURE,
       count(distinct claim_number) as clm_cnt,
       count(distinct patient_id) as pat_cnt
  FROM Dex_cataract_raven_proc
GROUP by 1
order by 2 desc

Query started at 02:26:13 PM India Standard Time; Query executed in 0.06 m

procedure  clm_cnt  pat_cnt
0      66984  2013671   871142
1      66982   178384    99125
2      66986     8202     5152
3      66850     5490     3726
4      G9390     4360     2651
5      66852     2650     1820
6      66840     2557     1649
7      66940      719      504
8      66983      665      465
9      66920      265      192
10     65175      234      183
11     G9389       23       18
12       136        2        2

Getting counts of patients, claim no. on the basis of procedures

In [24]:
%%read_sql
  SELECT COUNT(*), 
         COUNT(distinct patient_id) as pat_cnt, 
         COUNT(distinct claim_number) as clm_cnt
    FROM DEX_CATARACT_RAVEN_PROC 

Query started at 02:58:22 PM India Standard Time; Query executed in 0.09 m

COUNT(*)  pat_cnt  clm_cnt
0   2289095   944525  2212604

In [28]:
%%read_sql
    SELECT COUNT(CLAIM_NUMBER), COUNT(DISTINCT CLAIM_NUMBER)
      FROM DEX_CATARACT_RAVEN_PROC
     WHERE PLACE_SERVICE IS NULL 

Query started at 03:02:35 PM India Standard Time; Query executed in 0.06 m

COUNT(CLAIM_NUMBER)  COUNT(DISTINCT CLAIM_NUMBER)
0               608832                        563030

In [ ]:
%%read_sql

CREATE or replace TABLE Dex_cataract_raven_proc_1 as
   SELECT CLAIM_NUMBER,
          patient_suffix,
          ENCRYPTED_KEY_2,
          Patient_ID,

# RAVEN HEADER
Pulling additional fields from header for claims of interest

In [87]:
%%read_sql
DROP TABLE IF EXISTS EYEPOINT_CATARACT_PROC_HEAD;
CREATE TABLE EYEPOINT_CATARACT_PROC_HEAD as
      SELECT A.*,
             B.CLAIM_TYPE_CODE,
             B.TYPE_BILL
        FROM Eyepoint_cataract_raven_med A
             LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER B
                    ON A.claim_number = B.claim_number;

Query started at 07:20:22 PM India Standard Time; Query executed in 0.04 mQuery started at 07:20:24 PM India Standard Time; Query executed in 2.15 m

status
0  Table EYEPOINT_CATARACT_PROC_HEAD successfully...

In [29]:
%%read_sql

CREATE or replace TABLE DEX_CATARACT_RAVEN_PROC_HEAD as
      SELECT A.*,
             B.CLAIM_TYPE_CODE,
             B.TYPE_BILL
        FROM DEX_CATARACT_RAVEN_PROC A
             LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER B
                    ON A.claim_number = B.claim_number;

Query started at 03:05:07 PM India Standard Time; Query executed in 3.23 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD successfull...

In [33]:
%%read_sql
    SELECT COUNT(*), 
           COUNT(DISTINCT PATIENT_ID) as Pat_cnt,
           COUNT(DISTINCT claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD;

Query started at 03:31:53 PM India Standard Time; Query executed in 0.15 m

COUNT(*)  pat_cnt  clm_cnt
0   2334109   944525  2212604

## POS mapping

In [34]:
%%read_sql
ALTER TABLE DEX_CATARACT_RAVEN_PROC_HEAD
ADD POS VARCHAR();

Query started at 03:33:03 PM India Standard Time; Query executed in 0.05 m

status
0  Statement executed successfully.

In [35]:
%%read_sql
BEGIN;
    UPDATE DEX_CATARACT_RAVEN_PROC_HEAD
       SET POS = CASE WHEN (UPPER(CLAIM_TYPE_CODE) = 'P' AND PLACE_SERVICE = '24') 
                           OR UPPER(CLAIM_TYPE_CODE) = 'I' AND LEFT(TYPE_BILL,2) = '83' 
                              THEN 'ambulatory surgery center'
                      WHEN (UPPER(CLAIM_TYPE_CODE) = 'P' AND PLACE_SERVICE IN ('19','22')
                           OR UPPER(CLAIM_TYPE_CODE) = 'I' AND LEFT(TYPE_BILL,2) = '13')
                              THEN 'hospital outpatient'
                      ELSE 'other'
                   END;
COMMIT;

Query started at 03:33:44 PM India Standard Time; Query executed in 0.05 mQuery started at 03:33:47 PM India Standard Time; Query executed in 0.15 mQuery started at 03:33:56 PM India Standard Time; Query executed in 0.04 m

status
0  Statement executed successfully.

In [36]:
%%read_sql
    SELECT POS,
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD
  GROUP BY POS
 ORDER BY clm_cnt desc;

Query started at 03:34:38 PM India Standard Time; Query executed in 0.13 m

pos  pat_cnt  clm_cnt
0  ambulatory surgery center   509309  1129581
1        hospital outpatient   318702   681911
2                      other   192603   401670

In [37]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD

Query started at 03:35:11 PM India Standard Time; Query executed in 0.05 m

COUNT(*)  pat_cnt  clm_cnt
0   2334109   944525  2212604

# RAVEN PROVIDER

In [45]:
%%read_sql

CREATE or replace TABLE DEX_CATARACT_RAVEN_PROC_HEAD_PROV as
    SELECT A.*,
           B.PROVIDER_TYPE,
           B.PROVIDER_NPI,
           B.ENTITY_TYPE_CODE
  FROM DEX_CATARACT_RAVEN_PROC_HEAD A
       LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER B
              ON A.claim_number = B.claim_number
 WHERE LOWER(B.PROVIDER_TYPE) IN ('attending','rendering','operating')
 AND ENTITY_TYPE_CODE = 1;

Query started at 04:10:03 PM India Standard Time; Query executed in 5.38 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PROV succes...

In [47]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PROV

Query started at 04:16:08 PM India Standard Time; Query executed in 0.05 m

COUNT(*)  pat_cnt  clm_cnt
0   2266098   833840  1737414

In [46]:
%%read_sql
--One provider per claim
CREATE OR REPLACE TABLE DEX_CATARACT_RAVEN_PROC_HEAD_Phy as 
     SELECT A.*,
            case when b.provider_NPI is null then
            case when c.Provider_NPI is null then 
            d.Provider_NPI else c.Provider_NPI end
            else b.provider_NPI end as Physician_NPI
            
       FROM DEX_CATARACT_RAVEN_PROC_HEAD_PROV A
            LEFT JOIN (select distinct claim_number,Provider_NPI 
                         from DEX_CATARACT_RAVEN_PROC_HEAD_PROV 
                        where lower(provider_type) = 'attending' 
                              AND ENTITY_TYPE_CODE = 1) as b
                   on a.claim_number = b.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from DEX_CATARACT_RAVEN_PROC_HEAD_PROV 
                        where lower(provider_type) = 'rendering' 
                              AND ENTITY_TYPE_CODE = 1) as c
                   on a.claim_number = c.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from DEX_CATARACT_RAVEN_PROC_HEAD_PROV 
                        where lower(provider_type) = 'operating' 
                              AND ENTITY_TYPE_CODE = 1) as d
                   on a.claim_number = d.claim_number

Query started at 04:15:26 PM India Standard Time; Query executed in 0.12 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PHY success...

In [48]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY

Query started at 04:16:59 PM India Standard Time; Query executed in 0.10 m

COUNT(*)  pat_cnt  clm_cnt
0   2266434   833840  1737414

# Getting facility NPI from Maddy's table

In [49]:
%%read_sql

Create or replace table DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac as 
   SELECT A.*,
          B.organization_npi as facility_npi,
          B.organization_label
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY A
      LEFT JOIN PROJECT_ANALYTICS.REFERENCE.claims_submits_facility_master_new B
             on A.claim_number = B.claim_number
             AND lower(organization_label) not in ('practitioner','physician')

Query started at 04:17:37 PM India Standard Time; Query executed in 1.22 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PHY_FAC suc...

In [51]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as clm_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac  -- 1737414
     WHERE facility_npi is null            --200169

Query started at 04:22:54 PM India Standard Time; Query executed in 0.07 m

COUNT(*)  pat_cnt  clm_cnt
0    461077   110343   200169

# RAVEN PAYER
Pulling data from payer table to exclude medicare part from Raven

In [52]:
%%read_sql

CREATE or replace TABLE DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay as
    SELECT A.*,
           B.PAYER_ID,
           B.TYPE_COVERAGE
  FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac A
       LEFT JOIN RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PAYER B
              ON A.claim_number = B.claim_number;

Query started at 04:23:26 PM India Standard Time; Query executed in 3.95 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PHY_FAC_PAY...

## Type of coverage mapping

In [53]:
%%read_sql

CREATE TABLE DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_1 as
      SELECT A.*,
             C.MAP1 as TOC
        FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay A
             LEFT Join project_analytics.Pfizer.Plan_Type_Mapping_Options C
                    on A.TYPE_COVERAGE = C.TYPE_OF_COVERAGE;

Query started at 04:34:57 PM India Standard Time; Query executed in 0.22 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PHY_FAC_PAY...

In [4]:
%%read_sql

SELECT * FROM project_analytics.Pfizer.Plan_Type_Mapping_Options

Query started at 07:51:13 PM India Standard Time; Query executed in 0.06 m

type_of_coverage                                          plan_type  \
0                17                                    Other Plan Type   
1               COM                                         Commercial   
2              Null                                            Unknown   
3               MPD                          MEDICARE PART D Advantage   
4               MED                                      MEDICAID PLAN   
5               INT                                INTERNAL HOUSE PLAN   
6               DME                     DURABLE MEDICAL EQUIPMENT PLAN   
7              SPAP            STATE PHARMACEUTICAL ASSISTANCE PROGRAM   
8               MCM                         MANAGED CARE MEDICAID PLAN   
9               GOV                                         GOVERNMENT   
10            STATE                                     STATE PAYMENTS   
11              CSH                                               CASH   
12             ADAP                       AIDS DRUG ASSISTANCE PROGRAM   
13              MPB                               MEDICARE PART B PLAN   
14              SBA                   SMALL BUSINESS ADJUDICATION PLAN   
15             MPAB                              MEDICARE PART A AND B   
16              ELG                                   ELIGIBILITY PLAN   
17                1                                  LIABILITY MEDICAL   
18                2  HEALTH MAINTENANCE ORGANIZATION (HMO) MEDICARE...   
19                3                                 AUTOMOBILE MEDICAL   
20                4                                   MUTUALLY DEFINED   
21                5                                         DISABILITY   
22                9                                           SELF-PAY   
23               10                              CENTRAL CERTIFICATION   
24               11                         OTHER NON-FEDERAL PROGRAMS   
25               12              PREFERRED PROVIDER ORGANIZATION (PPO)   
26               13                             POINT OF SERVICE (POS)   
27               14              EXCLUSIVE PROVIDER ORGANIZATION (EPO)   
28               15                                INDEMNITY INSURANCE   
29               16  HEALTH MAINTENANCE ORGANIZATION (HMO) MEDICARE...   
30                A                                           SELF-PAY   
31               AM                                 AUTOMOBILE MEDICAL   
32                B                  WORKERS COMPENSATION HEALTH CLAIM   
33               BL                             BLUE CROSS/BLUE SHIELD   
34                C                                    MEDICARE PART A   
35               CH                                            CHAMPUS   
36               CI                           COMMERCIAL INSURANCE CO.   
37                D                                           MEDICAID   
38               DS                                         DISABILITY   
39                E                              OTHER FEDERAL PROGRAM   
40                F                           COMMERCIAL INSURANCE CO.   
41                G                             BLUE CROSS/BLUE SHIELD   
42                H                                            CHAMPUS   
43               HM                    HEALTH MAINTENANCE ORGANIZATION   
44                K                              CENTRAL CERTIFICATION   
45               LI                                          LIABILITY   
46               LM                                  LIABILITY MEDICAL   
47               MA                                    MEDICARE PART A   
48               MB                                    MEDICARE PART B   
49               MC                                           MEDICAID   
50               OF                              OTHER FEDERAL PROGRAM   
51                Q                             POINT OF SERVICE (POS)   
52                R              EXCLUSIVE PROVIDER ORGANIZATION (EPO)   
53     

In [54]:
%%read_sql
    SELECT count(*),
           count(distinct patient_id) as pat_cnt,
           count(distinct claim_number) as cml_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_1

Query started at 04:35:35 PM India Standard Time; Query executed in 0.08 m

COUNT(*)  pat_cnt  cml_cnt
0   3171379   833840  1737414

Checking counts of patients and claims per toc

In [55]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_1
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 04:36:41 PM India Standard Time; Query executed in 0.05 m

toc  pat_cnt  clm_cnt
0          Commercial   541229  1058424
1        Medicare FFS   342734   666647
2  Medicare Advantage    52500   123862
3              Others    50833    83651
4            Medicaid    38523    69460
5                None    32330    47151

## Removing Medicare toc

In [56]:
%%read_sql
CREATE OR REPLACE TABLE DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_2 as
     SELECT DISTINCT *
       FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_1
      WHERE lower(TOC) <> 'medicare ffs'
            OR lower(TOC) is null;

Query started at 04:43:25 PM India Standard Time; Query executed in 0.27 m

status
0  Table DEX_CATARACT_RAVEN_PROC_HEAD_PHY_FAC_PAY...

In [57]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) as Pat_cnt,
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt,
           COUNT(DISTINCT PROVIDER_NPI) as pro_cnt
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_2;

Query started at 04:43:59 PM India Standard Time; Query executed in 0.10 m

pat_cnt  clm_cnt  pro_cnt
0   654417  1354565    20363

In [58]:
%%read_sql
    SELECT TOC,
           COUNT(DISTINCT PATIENT_ID) AS PAT_CNT,
           COUNT(DISTINCT CLAIM_NUMBER) AS CLM_CNT
      FROM DEX_CATARACT_RAVEN_PROC_HEAD_PHY_fac_pay_2
     GROUP BY TOC
     ORDER BY CLM_CNT DESC;

Query started at 04:44:09 PM India Standard Time; Query executed in 0.07 m

toc  pat_cnt  clm_cnt
0          Commercial   541229  1058424
1  Medicare Advantage    52500   123862
2              Others    50833    83651
3            Medicaid    38523    69460
4                None    32330    47151

In [ ]:
---done till here---

# Normalizing Raven Provider Table

In [21]:
%%read_sql
Drop Table if exists Eyepoint_cataract_Raven_fac_phy;
Create table Eyepoint_cataract_Raven_fac_phy as
      SELECT distinct PATIENT_ID,
                      CLAIM_NUMBER,
                      YEAR_OF_SERVICE,
                      PROVIDER_TYPE,
                      PROVIDER_NPI,
                      ENTITY_TYPE_CODE,
                      TOC,
                      POS            
        FROM EYEPOINT_CATARACT_PROC_HEAD_DIAG_PROV_PAY_2;

Query started at 03:05:01 AM India Standard Time; Query executed in 0.10 mQuery started at 03:05:07 AM India Standard Time; Query executed in 0.13 m

status
0  Table EYEPOINT_CATARACT_RAVEN_FAC_PHY successf...

In [22]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) as Pat_cnt,
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt,
           COUNT(DISTINCT PROVIDER_NPI) as pro_cnt
      FROM Eyepoint_cataract_Raven_fac_phy;

Query started at 03:05:15 AM India Standard Time; Query executed in 0.08 m

pat_cnt  clm_cnt  pro_cnt
0   733257  1633710    35013

One provider per claim

In [23]:
%%read_sql
--One provider per claim
CREATE OR REPLACE TABLE Eyepoint_cataract_fac_phy_1 as 
     SELECT A.*,
            case when b.provider_NPI is null then
            case when c.Provider_NPI is null then 
            d.Provider_NPI else c.Provider_NPI end
            else b.provider_NPI end as Physician_NPI,
            
            case when e.provider_NPI is null then
            case when f.Provider_NPI is null then 
            g.Provider_NPI else f.Provider_NPI end
            else e.provider_NPI end as Facility_NPI
            
       FROM Eyepoint_cataract_Raven_fac_phy A
            LEFT JOIN (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY 
                        where lower(provider_type) = 'attending' 
                              AND ENTITY_TYPE_CODE = 1) as b
                   on a.claim_number = b.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY 
                        where lower(provider_type) = 'rendering' 
                              AND ENTITY_TYPE_CODE = 1) as c
                   on a.claim_number = c.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY 
                        where lower(provider_type) = 'operating' 
                              AND ENTITY_TYPE_CODE = 1) as d
                   on a.claim_number = d.claim_number
            LEFT JOIN (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY  
                        where lower(provider_type) = 'facility' 
                              AND ENTITY_TYPE_CODE = 2) as e
                   on a.claim_number = e.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY  
                        where lower(provider_type) = 'billing' 
                              AND ENTITY_TYPE_CODE = 2) as f
                   on a.claim_number = f.claim_number
            left join (select distinct claim_number,Provider_NPI 
                         from EYEPOINT_CATARACT_RAVEN_FAC_PHY  
                        where lower(provider_type) = 'servicelocation' 
                              AND ENTITY_TYPE_CODE = 2) as g
                   on a.claim_number = g.claim_number;

Query started at 03:05:19 AM India Standard Time; Query executed in 0.14 m

status
0  Table EYEPOINT_CATARACT_FAC_PHY_1 successfully...

In [24]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) as Pat_cnt,
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt,
           COUNT(DISTINCT PROVIDER_NPI) as pro_cnt,
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt,
           COUNT(DISTINCT PHYSICIAN_NPI) as phy_cnt
      FROM Eyepoint_cataract_fac_phy_1;

Query started at 03:05:28 AM India Standard Time; Query executed in 0.09 m

pat_cnt  clm_cnt  pro_cnt  fac_cnt  phy_cnt
0   733257  1633710    35013    12364    19742

In [9]:
%%read_sql

SELECT * FROM Eyepoint_cataract_fac_phy_1
WHERE facility_npi = '1003842790' and entity_type_code = 2

Query started at 02:57:09 PM India Standard Time; Query executed in 0.05 m

patient_id           claim_number  \
0   Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EP061518723945255   
1   4sNZQCP2UXAMDEwV2aq/1/ML6+s7lmHzp29p0bdcJTM=v2...  con_EP101317739126696   
2   BGN8aTVdST8k7N/E2dtKFf2NVUX05ThzDzyvTFD6W5g=9q...  con_EP060618788830349   
3   BGN8aTVdST8k7N/E2dtKFf2NVUX05ThzDzyvTFD6W5g=9q...  con_EP062518755701161   
4   BGN8aTVdST8k7N/E2dtKFf2NVUX05ThzDzyvTFD6W5g=9q...  con_EP062518755701161   
5   Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EP061518723945255   
6   Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EI050818283318295   
7   Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EP050818783316603   
8   BGN8aTVdST8k7N/E2dtKFf2NVUX05ThzDzyvTFD6W5g=9q...  con_EP060618788830349   
9   Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EP050818783316603   
10  Ce6n+zn4BDxbI94qWft3QL14s71ZSmvvhFIdZRGtWFQ=TC...  con_EI050818283318295   
11  4sNZQCP2UXAMDEwV2aq/1/ML6+s7lmHzp29p0bdcJTM=v2...  con_EP101317739126696   

   year_of_service provider_type provider_npi  entity_type_code         toc  \
0       2018-04-05      Facility   1003842790                 2  Commercial   
1       2017-07-27      Facility   1003842790                 2  Commercial   
2       2017-12-28      Facility   1003842790                 2  Commercial   
3       2017-12-14      Facility   1003842790                 2  Commercial   
4       2017-12-14       Billing   1568411395                 2  Commercial   
5       2018-04-05       Billing   1568411395                 2  Commercial   
6       2018-03-06      Facility   1003842790                 2  Commercial   
7       2018-03-06       Billing   1568411395                 2  Commercial   
8       2017-12-28       Billing   1568411395                 2  Commercial   
9       2018-03-06      Facility   1003842790                 2  Commercial   
10      2018-03-06       Billing   1568411395                 2  Commercial   
11      2017-07-27       Billing   1568411395                 2  Commercial   

                          pos physician_npi facility_npi  
0   ambulatory surgery center    1740258904   1003842790  
1   ambulatory surgery center    1740258904   1003842790  
2   ambulatory surgery center    1740258904   1003842790  
3   ambulatory surgery center    1740258904   1003842790  
4   ambulatory surgery center    1740258904   1003842790  
5   ambulatory surgery center    1740258904   1003842790  
6         hospital outpatient    1396862900   1003842790  
7   ambulatory surgery center    1396862900   1003842790  
8   ambulatory surgery center    1740258904   1003842790  
9   ambulatory surgery center    1396862900   1003842790  
10        hospital outpatient    1396862900   1003842790  
11  ambulatory surgery center    1740258904   1003842790

# Raven + CMS OP Provider Tables
CMS tables are created in other notebook
PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1 
PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1

In [41]:
%%read_sql
DROP TABLE IF EXISTS EYEPOINT_CATARACT_FACILITY_PHYSICIAN;
CREATE TABLE EYEPOINT_CATARACT_FACILITY_PHYSICIAN AS
      SELECT DISTINCT 'raven' as SOURCE,
                      PATIENT_ID,
                      CLAIM_NUMBER,
                      lower(POS) as pos,
                      FACILITY_NPI,
                      PHYSICIAN_NPI
       FROM EYEPOINT_CATARACT_FAC_PHY_1
      UNION
     SELECT DISTINCT 'cms_op' as SOURCE,
                     CAST(PATIENT_ID AS STRING),
                     CAST(CLAIM_NUMBER AS STRING),
                     'hospital outpatient' as POS,
                     FACILITY_NPI,
                     PHYSICIAN_NPI 
       FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1;

Query started at 03:16:34 AM India Standard Time; Query executed in 0.07 mQuery started at 03:16:38 AM India Standard Time; Query executed in 0.17 m

status
0  Table EYEPOINT_CATARACT_FACILITY_PHYSICIAN suc...

In [26]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) as pat_cnt, 
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt, 
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt, 
           COUNT(DISTINCT PHYSICIAN_NPI) as phy_cnt
      FROM EYEPOINT_CATARACT_FACILITY_PHYSICIAN;

Query started at 03:05:44 AM India Standard Time; Query executed in 0.09 m

pat_cnt  clm_cnt  fac_cnt  phy_cnt
0  1093028  2180071    12611    20884

In [27]:
%%read_sql
    SELECT SOURCE,
           COUNT(DISTINCT PATIENT_ID) as pat_cnt, 
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt, 
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt, 
           COUNT(DISTINCT PHYSICIAN_NPI) as phy_cnt
      FROM EYEPOINT_CATARACT_FACILITY_PHYSICIAN
     GROUP BY SOURCE
     ORDER BY clm_cnt DESC;

Query started at 03:05:49 AM India Standard Time; Query executed in 0.07 m

source  pat_cnt  clm_cnt  fac_cnt  phy_cnt
0   raven   733257  1633710    12364    19742
1  cms_op   359771   546361     2606     7963

In [28]:
%%read_sql
    SELECT 'cms_asc' as source,
           sum(ALLOWED_SERVICES) as clm_cnt,
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt 
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1

Query started at 03:05:54 AM India Standard Time; Query executed in 0.05 m

source  clm_cnt  fac_cnt
0  cms_asc  1339985     1926

In [29]:
%%read_sql
GRANT ALL on table EYEPOINT_CATARACT_FACILITY_PHYSICIAN to role RWD_ANALYTICS_RW

Query started at 03:05:57 AM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

## All Facilities

In [30]:
%%read_sql
DROP TABLE IF EXISTS ALL_FACILITIES;
CREATE TABLE ALL_FACILITIES AS
    SELECT DISTINCT FACILITY_NPI
      FROM EYEPOINT_CATARACT_FACILITY_PHYSICIAN
     WHERE FACILITY_NPI IS NOT NULL 
     UNION
    SELECT DISTINCT FACILITY_NPI
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1
     WHERE FACILITY_NPI IS NOT NULL ;

Query started at 03:06:01 AM India Standard Time; Query executed in 0.06 mQuery started at 03:06:04 AM India Standard Time; Query executed in 0.12 m

status
0  Table ALL_FACILITIES successfully created.

In [ ]:
%%read_sql Unique_facilities -d
SELECT distinct * from ALL_FACILITIES;

In [57]:
Unique_facilities.to_excel('Unique_facilities.xlsx',index=False)

In [31]:
%%read_sql
    SELECT COUNT(DISTINCT FACILITY_NPI) FROM ALL_FACILITIES;

Query started at 03:06:11 AM India Standard Time; Query executed in 0.07 m

COUNT(DISTINCT FACILITY_NPI)
0                         12744

## Facility info from healthbase

In [32]:
%%read_sql
DROP TABLE if exists Eyepoint_healthbase_facility_map;
CREATE TABLE Eyepoint_healthbase_facility_map as 
      SELECT FACILITY_NPI,
             Healthbase_ID
        FROM ALL_FACILITIES A 
             LEFT JOIN rwd_db.RWD.HEALTHBASE_ENTITYLOOKUP B
                    ON A.FACILITY_NPI = B.identifier;

Query started at 03:06:15 AM India Standard Time; Query executed in 0.06 mQuery started at 03:06:19 AM India Standard Time; Query executed in 0.13 m

status
0  Table EYEPOINT_HEALTHBASE_FACILITY_MAP success...

In [33]:
%%read_sql  
drop table if exists Eyepoint_healthbase_facility_detail_hb;
Create Table Eyepoint_healthbase_facility_detail_hb as 
      select A.*,
             B.NAME,
             B.ADDRESS_LINE_1 AS ADDRESS,
             B.CITY,
             B.STATE,
             B.ZIP_CODE 
        from Eyepoint_healthbase_facility_map A
             left Join rwd_db.RWD.HEALTHBASE_ENTITYSUMMARY B
                    on A.Healthbase_ID = B.Healthbase_ID;

Query started at 03:06:27 AM India Standard Time; Query executed in 0.06 mQuery started at 03:06:30 AM India Standard Time; Query executed in 0.11 m

status
0  Table EYEPOINT_HEALTHBASE_FACILITY_DETAIL_HB s...

In [34]:
%%read_sql
    SELECT COUNT(DISTINCT FACILITY_NPI) FROM Eyepoint_healthbase_facility_detail_hb
    WHERE NAME IS NULL

Query started at 03:06:37 AM India Standard Time; Query executed in 0.06 m

COUNT(DISTINCT FACILITY_NPI)
0                            78

## Remaining Facility info from CMS

In [35]:
%%read_sql
drop table if exists Eyepoint_healthbase_facility_detail_cms;
Create Table Eyepoint_healthbase_facility_detail_cms as 
      Select npi as FACILITY_NPI,  
             PROVIDER_ORGANIZATION_NAME_LEGAL_BUSINESS_NAME as NAME,
             PROVIDER_FIRST_LINE_BUSINESS_PRACTICE_LOCATION_ADDRESS AS ADDRESS,
             PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_CITY_NAME AS CITY,
             PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_STATE_NAME AS STATE,
             PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_POSTAL_CODE AS ZIP_CODE
        from RWD_DB.RWD.THIRD_PARTY_CMSNPI
       where npi in (SELECT DISTINCT FACILITY_NPI FROM Eyepoint_healthbase_facility_detail_hb
                      WHERE NAME IS NULL);

Query started at 03:06:41 AM India Standard Time; Query executed in 0.08 mQuery started at 03:06:46 AM India Standard Time; Query executed in 0.16 m

status
0  Table EYEPOINT_HEALTHBASE_FACILITY_DETAIL_CMS ...

In [36]:
%%read_sql
    SELECT COUNT(DISTINCT FACILITY_NPI) FROM Eyepoint_healthbase_facility_detail_cms
    WHERE NAME IS NULL

Query started at 03:06:55 AM India Standard Time; Query executed in 0.09 m

COUNT(DISTINCT FACILITY_NPI)
0                             0

## Facility Info HB + CMS

In [61]:
%%read_sql
DROP TABLE IF EXISTS Eyepoint_healthbase_facility_detail;
CREATE TABLE Eyepoint_healthbase_facility_detail AS
      SELECT DISTINCT FACILITY_NPI,
                      NAME,
                      ADDRESS,
                      CITY,
                      STATE,
                      ZIP_CODE
        FROM Eyepoint_healthbase_facility_detail_HB
       WHERE NAME IS NOT NULL 
       UNION
      SELECT DISTINCT FACILITY_NPI,
                      NAME,
                      ADDRESS,
                      CITY,
                      STATE,
                      ZIP_CODE
        FROM Eyepoint_healthbase_facility_detail_CMS;

Query started at 03:48:33 PM India Standard Time; Query executed in 0.04 mQuery started at 03:48:36 PM India Standard Time; Query executed in 0.07 m

status
0  Table EYEPOINT_HEALTHBASE_FACILITY_DETAIL succ...

In [62]:
%%read_sql
select count(distinct FACILITY_NPI), count(distinct NAME)
from Eyepoint_healthbase_facility_detail

Query started at 03:49:08 PM India Standard Time; Query executed in 0.09 m

COUNT(DISTINCT FACILITY_NPI)  COUNT(DISTINCT NAME)
0                         12744                 11551

In [63]:
%%read_sql FAC_NAME
    SELECT DISTINCT FACILITY_NPI, NAME 
      FROM Eyepoint_healthbase_facility_detail
     ORDER BY NAME;

Query started at 03:49:14 PM India Standard Time; Query executed in 0.17 m

facility_npi                                               name
0        1942403969           (CLOSED) EYE SURGERY CENTER OF MIDDLE TN
1        1578591756                   (CLOSED) MARSHALL MEDICAL CENTER
2        1366489189                                 20 20 EYECARE, PSC
3        1205006079                                  20 20 Eyecare Inc
4        1710061593                                 20 20 Eyecare Inc.
5        1104957257                           20 20 Vision Clinic, Llp
6        1083652978                                  20-20 EYECARE PSC
7        1548207434                                  20-20 EYECARE PSC
8        1073550927                                  20-20 EYECARE PSC
9        1821507518                                      20-20 Eyecare
10       1366520702                                 20-20 Eyecare, Psc
11       1295772143                                 20-20 Eyecare, Psc
12       1710012190                                 20-20 Eyeworld Inc
13       1922350636                           20-20 Vision Express Llc
14       1821139932                                  20-20 Visions Llc
15       1134292915                       20/20 Eyecare Centers Inc Ps
16       1194718940                          20/20 SURGERY CENTER, LLC
17       1871588442                  2020 Eye Physicians Of Indiana Pc
18       1922141738                     25 EAST SAME DAY SURGERY CENTE
19       1417308149                             3 C Vision Specialists
20       1679980965                               3 Rivers Eyecare Llc
21       1003078718                       3 Visions Ophthalmology P.C.
22       1528175429                                        3-D Optical
23       1962676304                    32ND STREET SURGERY CENTER, LLC
24       1124214051                                      3F VISION LCC
25       1013283639                                      3F VISION LLC
26       1669668836                                      3F VISION LLC
27       1326474198                                      3F VISION LLC
28       1235326034                                      3F VISION LLC
29       1558640250                                      3F VISION LLC
30       1508083502                                      3F VISION LLC
31       1770866428                                      3F VISION LLC
32       1437329034                                   4 Your Eyes, Llc
33       1386036721                              417 Eyes A Series Llc
34       1902964174                     4TH STREET LASER & SURGERY CTR
35       1912399841                                    4Ureye Care Llc
36       1801259734                                      605 Vision Pc
37       1326176249                           8Th Ave Ophthalmology Pc
38       1295769131                  90210 SURGERY MEDICAL CENTER, LLC
39       1588647770                                 96TH MEDICAL GROUP
40       1356505663                           A & A Eye Associates, Pc
41       1356549349                                  A & D Medical, Pc
42       1538613849                                  A & E EYECARE INC
43       1003005356                        A Better Vue Eye Physicians
44       1912058728  A Center For Visioncare Surgical And Medical G...
45       1891912846                    A New Concept Optical & Eyecare
46       1801289269                      A Richard Cote Md Corporation
47       1932375417                        A William G Schubert Md Ltd
48       1205140126                             A. Dylan Reach, Od, Pc
49       1225047509  A. Ebbie Soroudi, Md, Ms A Professional Medica...
50       1669672044                               A. R. Copely, Jr, Pa
51       1194947200                   A.M. Palanca-Capistrano,M.D. Inc
52       1386861649                        A.Terrel Williams, Md, Pllc
53       1215227111                                     A2 Optical Llc
54       1477760981                                  ABBA EYE CARE INC
55       1114132644            

In [64]:
FAC_NAME.to_excel('FAC_NAME.xlsx',index=False)